In [ ]:
# 231113
# script for github

In [1]:
import pandas as pd

In [2]:
# for the comparisons from the original Orthofinder outputs
main_path = r"/path_to_files"

### function to count number of genes per species in orthogroups

In [3]:
def countGenes(OGfile, program, startNum):
    
    orthogroup = pd.read_table(OGfile)
    print("orthogroup shape:", orthogroup.shape)
    OGcount = orthogroup.fillna("*")
    
    if program == "orthofinder":
        for row, value in OGcount.iterrows():
            index = OGcount.shape[1]

            # if start with Phylogenetically Hieirachical Orthogroups, start with i=3
            # if start with Orthogroups (no downstream analyses), start with i=1
            i=startNum
            while i < index:
                items = value[i]
                #print(orthogroup.iloc[row][i])
                if type(items) != int:
                    geneList = items.split(", ")
                    #print(geneList)
                    if geneList[0] == "*":
                        OGcount.iat[row,i] = 0
                        i += 1
                    else:
                        numOfGenes = len(geneList)
                        OGcount.iat[row,i] = numOfGenes
                        i += 1
                else:
                    i += 1
                    
    elif program == "sonicparanoid":
        
        OGcount.columns = [col_name.split(".")[0] for col_name in OGcount.columns]
        
        for row, value in OGcount.iterrows():
            index = OGcount.shape[1]
            
            i=1
            while i < index:
                items = value[i]
                #print(orthogroup.iloc[row][i])
                if type(items) != int:
                    geneList = items.split(",")
                    #print(geneList)
                    if geneList[0] == "*":
                        OGcount.iat[row,i] = 0
                        i += 1
                    else:
                        numOfGenes = len(geneList)
                        OGcount.iat[row,i] = numOfGenes
                        i += 1
                else:
                    i += 1
                    
    elif program == "broccoli":
        OGcount.columns = [col_name.split(".")[0] for col_name in OGcount.columns]
        OGcount.rename(columns={"#OG_name": "OG_name"}) #230124 change, help with visualization R script later 
        
        for row, value in OGcount.iterrows():
            index = OGcount.shape[1]

            i=1
            while i < index:
                items = value[i]
                #print(orthogroup.iloc[row][i])
                if type(items) != int:
                    geneList = items.split(" ")
                    #print(geneList)
                    if geneList[0] == "*":
                        OGcount.iat[row,i] = 0
                        i += 1
                    else:
                        numOfGenes = len(geneList)
                        OGcount.iat[row,i] = numOfGenes
                        i += 1
                else:
                    i += 1
   
    elif program == "orthnet":
        for row, value in OGcount.iterrows():
            index = OGcount.shape[1]
            
            # if start with Orthogroups (no downstream analyses), start with i=1
            i=startNum
            while i < index:
                items = value[i]
                #print(orthogroup.iloc[row][i])
                if type(items) != int:
                    geneList = items.split(", ")
                    #print(geneList)
                    if geneList[0] == "*":
                        OGcount.iat[row,i] = 0
                        i += 1
                    else:
                        numOfGenes = len(geneList)
                        OGcount.iat[row,i] = numOfGenes
                        i += 1
                else:
                    i += 1
                    
    return(OGcount)
    #OGcount.to_csv(outputPath, sep = "\t", index=False)

### function to convert the number of genes per species into a binary code 

In [4]:
# 0 = no genes
# 1 = at least 1 gene

def binaryGenes(OGdf):
    
    print("orthogroup shape:", OGdf.shape)
    
    for row, value in OGdf.iterrows():
        index = OGdf.shape[1]
    
        i=1
        while i < index:
            numGenes = value[i]
            #print(orthogroup.iloc[row][i])
            if numGenes > 1:
                OGdf.iat[row,i] = 1
                i += 1
            else:
                i += 1
            

    return(OGdf)

### function to keep the number of genes per species in each orthogroup

In [5]:
# but limit only to numbers below 10
# numbers 10 and > 10 will be noted as "10"
def limitNumGenes(OGdf):
    
    print("orthogroup shape:", OGdf.shape)
    for row, value in OGdf.iterrows():
        index = OGdf.shape[1]
    
        i=1
        while i < index:
            numGenes = value[i]
            #print(orthogroup.iloc[row][i])
            if numGenes > 10:
                OGdf.iat[row,i] = 10
                i += 1
            else:
                i += 1
                
    return(OGdf)

### wrapper functions to run all of the conversion functions and write to files

In [6]:
# output directory to "diploidBrass" for diploid set
def OG_to_convert_d(OGfile, program, colNum, outputDir):
    countDF = countGenes(OGfile, program, colNum)
    if program == "orthofinder":
        countDF = countDF.drop(columns=['OG', 'Gene Tree Parent Clade'])
    dfCopy1 = countDF.copy()
    binDF = binaryGenes(dfCopy1)
    dfCopy2 = countDF.copy() 
    tenMaxDF = limitNumGenes(dfCopy2)
    output_path = r"/path_for_diploid_output/diploidBrass"
    countDF.to_csv(output_path + outputDir + "/" + program + "_allCounts.tsv", sep="\t", encoding='utf-8', index=False)
    binDF.to_csv(output_path + outputDir + "/" + program + "_binary.tsv", sep="\t", encoding='utf-8', index=False)
    tenMaxDF.to_csv(output_path + outputDir + "/" + program + "_tenMax.tsv", sep="\t", encoding='utf-8', index=False)

# output directory to "fullBrass" for higher ploidy set
def OG_to_convert_f(OGfile, program, colNum, outputDir):
    countDF = countGenes(OGfile, program, colNum)
    if program == "orthofinder":
        countDF = countDF.drop(columns=['OG', 'Gene Tree Parent Clade'])
    dfCopy1 = countDF.copy()
    binDF = binaryGenes(dfCopy1)
    dfCopy2 = countDF.copy() 
    tenMaxDF = limitNumGenes(dfCopy2)
    output_path = r"/path_for_full_output/fullBrass"
    countDF.to_csv(output_path + outputDir + "/" + program + "_allCounts.tsv", sep="\t", encoding='utf-8', index=False)
    binDF.to_csv(output_path + outputDir + "/" + program + "_binary.tsv", sep="\t", encoding='utf-8', index=False)
    tenMaxDF.to_csv(output_path + outputDir + "/" + program + "_tenMax.tsv", sep="\t", encoding='utf-8', index=False)


### Broccoli

In [ ]:
BR_path_d = main_path + "/path_to_dip/dip_BR-table_OGs_protein_names.txt"
BR_path_f = main_path + "/path_to_full/full_BR-table_OGs_protein_names.txt"

OG_to_convert_d(BR_path_d, "broccoli", 1, "BR")
OG_to_convert_f(BR_path_f, "broccoli", 1, "BR")

### Orthofinder-BLAST

In [ ]:
OF_b_path_d = main_path + "/path_to_dip/dip_OF_b_N0.tsv"
OF_b_path_f = main_path + "/path_to_full/full_OFb_N0.tsv"

OG_to_convert_d(OF_d_path_d, "orthofinder", 3, "OF_blast")
OG_to_convert_f(OF_d_path_f, "orthofinder", 3, "OF_blast")

### Orthofinder-DIAMOND

In [ ]:
OF_d_path_d = main_path + "/path_to_dip/dip_OF_d_N0.tsv"
OF_d_path_f = main_path + "/path_to_full/full_OFd_N0.tsv"

OG_to_convert_d(OF_d_path_d, "orthofinder", 3, "OF_diamond")
OG_to_convert_f(OF_d_path_f, "orthofinder", 3, "OF_diamond")

### Orthofinder-MMseqs2

In [ ]:
OF_m_path_d = main_path + "/path_to_dip/dip_OF_m_N0.tsv"
OF_m_path_f = main_path + "/path_to_full/full_OFm_N0.tsv"

OG_to_convert_d(OF_m_path_d, "orthofinder", 3, "OF_mmseqs")
OG_to_convert_f(OF_m_path_f, "orthofinder", 3, "OF_mmseqs")

### SonicParanoid-DIAMOND

In [ ]:
SP_d_path_d = main_path + "/path_to_dip/dip_SP_d-flat.ortholog_groups.tsv"
SP_d_path_f = main_path + "/path_to_full/full_SPd_flat.ortholog_groups.tsv"

OG_to_convert_d(SP_d_path_d, "sonicparanoid", 1, "SP_diamond")
OG_to_convert_f(SP_d_path_f, "sonicparanoid", 1, "SP_diamond")

### SonicParanoid-MMseqs2

In [ ]:
SP_m_path_d = main_path + "/path_to_dip/dip_SP_m-flat.ortholog_groups.tsv"
SP_m_path_f = main_path + "/path_to_full/full_SPm-flat.ortholog_groups.tsv"

OG_to_convert_d(SP_m_path_d, "sonicparanoid", 1, "SP_mmseqs")
OG_to_convert_f(SP_m_path_f, "sonicparanoid", 1, "SP_mmseqs")

### OrthNet

In [ ]:
ON_path_d = main_path + "/path_to_dip/OrthNet_groups_diploid_230927.txt"
ON_path_f = main_path + "/path_to_full/OrthNet_groups_full_230915.txt"

OG_to_convert_d(ON_path_d, "orthnet", 1, "ON")
OG_to_convert_f(ON_path_f, "orthnet", 1, "ON")